In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



# Function to calculate rolling volatility
def calculate_volatility(returns_series, window_size):
    # Calculate the rolling standard deviation of returns
    return returns_series.rolling(window=window_size).std()


# Function to calculate mean volatility for each group of 5 rows
def calculate_mean_volatility(df):
    df['Group'] = df.index // 5
    group_means = df.groupby('Group')['volatility_ratio'].mean()
    df['mean_Volatility'] = np.nan

    for group in df['Group'].unique():
        last_row_index = df[df['Group'] == group].index[-1]
        df.at[last_row_index, 'mean_Volatility'] = group_means[group]
    
    return df


# Function to update the DataFrame with new entry
def update_dataframe(df, new_date, new_nifty_price):
    # Ensure the date column is sorted
    df.sort_values('date', inplace=True)
    
    # Calculate the daily return for the new price
    last_price = df.iloc[-1]['nifty200']
    daily_return = (new_nifty_price - last_price) / last_price
    
    # Append the new data to the DataFrame
    new_data = {
        'date': pd.to_datetime(new_date, format='%d-%m-%Y'),  # parsing the date string into a datetime object
        'nifty200': new_nifty_price,
        'daily_return': daily_return,
        'volatility_21_days': None,  # to be calculated
        'volatility_252_days': None,  # to be calculated
        'volatility_ratio': None  # to be calculated
    }
    df = df.append(new_data, ignore_index=True)
    
    # Calculate the rolling 21-day and 250-day volatilities
    df['volatility_21_days'] = calculate_volatility(df['daily_return'], 21)
    df['volatility_252_days'] = calculate_volatility(df['daily_return'], 252)
    
    # Calculate the ratio of the 21-day volatility to the 250-day volatility
    df['volatility_ratio'] = df['volatility_21_days'] / df['volatility_252_days']
    
    return df

# Example usage - After updating the DataFrame with new data
new_date = input("Enter date here (DD-MM-YYYY) : ")#'30-11-2023'   # New date in the format 'DD-MM-YYYY'
new_nifty_price = float(input("Enter nifty_200 value : " )) #10887.35  # New Nifty price as a float

# Load your CSV file into a DataFrame
df = pd.read_csv("nifty200_new.csv",parse_dates=['date'], dayfirst=True)

# Update DataFrame with new data
updated_df = update_dataframe(df, new_date, new_nifty_price)

# Calculate mean volatility for each group of 5 rows
updated_df = calculate_mean_volatility(updated_df)

updated_df.to_csv('nifty200_new.csv', index=False)

updated_df_2022 = updated_df[updated_df['date'].dt.year >= 2022]

# Plotting the graph
plt.figure(figsize=(10, 6))
plt.plot(updated_df_2022['date'], updated_df_2022['mean_Volatility'],marker = "o",linestyle='' )
plt.axhline(y=1.5, color='r', linestyle='--')
plt.title('Mean Volatility Over Of NIFTY 200 - Time(2022 - 2023)')
plt.xlabel('Date')
plt.ylabel('Mean Volatility')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability



plt.show()


In [ ]:
updated_df = pd.read_csv('nifty200_new.csv', parse_dates=['date'], dayfirst=True)

In [16]:

updated_df_2022 = updated_df[updated_df['date'].dt.year >= 2022]

In [17]:
updated_df_2022

,date,nifty200,daily_return,volatility_21_days,volatility_252_days,volatility_ratio,Group,mean_Volatility
3220,2022-01-03,9348.00,0.014064,0.011272,0.009663,1.166502,644,NaN
3221,2022-01-04,9422.45,0.007964,0.010607,0.009672,1.096590,644,NaN
3222,2022-01-05,9477.20,0.005811,0.010213,0.009676,1.055545,644,NaN
3223,2022-01-06,9405.40,-0.007576,0.009829,0.009691,1.014262,644,NaN
3224,2022-01-07,9442.65,0.003960,0.009843,0.009691,1.015673,644,1.069714
...,...,...,...,...,...,...,...,...
3684,2023-11-16,10651.85,0.004944,0.007732,0.006210,1.245102,736,1.188911
3685,2023-11-17,10649.50,-0.000221,0.007517,0.006210,1.210441,737,NaN
3686,2023-11-20,10631.80,-0.001662,0.007507,0.006209,1.209030,737,NaN
3687,2023-11-21,10667.35,0.003344,0.007380,0.006210,1.188354,737,NaN
